In [158]:
import numpy as np
import pandas as pd 
import os
import seaborn as sns
# Better display the output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.display.max_rows

500

In [159]:
os.chdir(r'G:\My Drive\שנה ב\שנה ב סמסטר ב\פרויקט דאטא\Analytics-Based Project_')
X = pd.read_csv("numeric_dataframe.csv")

In [160]:
table = X[["loan_amnt","total_pymnt","sub_grade","realised_return"]]

In [161]:
table = X[["loan_amnt","total_pymnt","sub_grade","realised_return"]]
grades_weighted_rate = {}
for i in np.unique(table.sub_grade) : 

    grade_total = table[table.sub_grade ==i].loan_amnt.sum()
    
    weighted_rate=(table[table.sub_grade ==i].loan_amnt/grade_total * table[table.sub_grade ==i].realised_return).sum()
    
    grades_weighted_rate[i] = weighted_rate
    
grades_weighted_rate 

{1.0: 0.028340086700283644,
 1.2: 0.028714582979716508,
 1.4: 0.027337767229036285,
 1.6: 0.0278688359584119,
 1.8: 0.027834634408985388,
 2.0: 0.02229091267014209,
 2.2: 0.024712850285545686,
 2.4: 0.024393444839394207,
 2.6: 0.02001683575276212,
 2.8: 0.015315980862835464,
 3.0: 0.012292859066544974,
 3.2: 0.009076562695580104,
 3.4: 0.010337729908573732,
 3.6: 0.0048568744716596755,
 3.8: 0.0018038429035520187,
 4.0: -0.005073077400830121,
 4.2: -0.005345585233379847,
 4.4: -0.011651263573327944,
 4.6: -0.009335753898913671,
 4.8: -0.01598580599740072,
 5.0: -0.01729885824162645,
 5.2: -0.022601410378018563,
 5.4: -0.025613070434736325,
 5.6: -0.038142403183034745,
 5.8: -0.03902079953812031,
 6.0: -0.03525566533061633,
 6.2: -0.0460715432825596,
 6.4: -0.024822700142614073,
 6.6: -0.04442386911930442,
 6.8: -0.07222303408574988,
 7.0: -0.03204750152538866,
 7.2: -0.0787920499500341,
 7.4: -0.04790342642163647,
 7.6: -0.06951437518082539,
 7.8: -0.10214215841934654}

In [162]:
# ok we see grade 2.6 is slighly higher then 2% so this is the criteria

total_grades_critira = table[table.sub_grade <= 2.6].loan_amnt.sum()

ROI_grade_crit = (table[table.sub_grade <= 2.6].loan_amnt/total_grades_critira * table[table.sub_grade <= 2.6].realised_return).sum()

ROI_grade_crit

0.025342590881713943

In [163]:
ROI_grade_crit

0.025342590881713943

In [167]:
X = X.drop(columns=["loan_lifetime_in_months","estimated_monthly_paymnent","total_pymnt","loan_status","last_pymnt_d"])

KeyError: "['loan_lifetime_in_months', 'estimated_monthly_paymnent', 'total_pymnt', 'loan_status', 'last_pymnt_d'] not found in axis"

In [87]:
from sklearn.model_selection import train_test_split

# Making sure the last column will hold the labels
X, y = X.drop('realised_return', axis=1), X['realised_return']

# Splitting the data into 60% train and 40% remaining_data
X_train, remaining_data, y_train, remaining_labels = train_test_split(X, y, test_size=0.4, random_state=99)

# Splitting the remaining_data and remaining_labels into 50% test and 50% validation
X_test, X_val, y_test, y_val = train_test_split(remaining_data, remaining_labels, test_size=0.5, random_state=99)


In [88]:
# we look to beat the grades showing more then 2% 


above_2per_grades_subset = pd.concat([X,y],axis=1).groupby("sub_grade").mean()["realised_return"]>= 0.02
ROI_grades = pd.concat([X,y],axis=1).groupby("sub_grade").mean()["realised_return"][above_2per_grades_subset].mean()
ROI_grades


0.025098014230550424

In [89]:
pd.concat([X,y],axis=1).groupby("sub_grade").mean()["realised_return"]

sub_grade
1.0    0.027949
1.2    0.027156
1.4    0.027128
1.6    0.027161
1.8    0.026416
2.0    0.022228
2.2    0.023974
2.4    0.023613
2.6    0.020257
2.8    0.015622
3.0    0.013589
3.2    0.009573
3.4    0.009819
3.6    0.006540
3.8    0.004439
4.0   -0.000559
4.2   -0.000811
4.4   -0.006967
4.6   -0.005223
4.8   -0.010424
5.0   -0.010383
5.2   -0.012850
5.4   -0.016475
5.6   -0.024960
5.8   -0.028732
6.0   -0.022810
6.2   -0.042102
6.4   -0.020322
6.6   -0.027347
6.8   -0.042184
7.0   -0.020282
7.2   -0.070777
7.4   -0.034620
7.6   -0.053891
7.8   -0.095384
Name: realised_return, dtype: float64

In [90]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error

In [91]:
# start of modeling process 

model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
net_ROI = y_test[y_pred >= 0.02].mean()

r2_score(y_val,y_pred)
print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


r^2: 0.022675953929285564 MAE: 0.06774788060792665 ROI lift: 1.1372803167844228 net roi: 0.016507334184799948


In [92]:
# trying min max normaliztion 
X_val2 = (X_val-X_val.min(axis=0))/(X_val.max(axis=0)-X_val.min(axis=0))
X_train2 = (X_train-X_train.min(axis=0))/(X_train.max(axis=0)-X_train.min(axis=0))

model2 = LinearRegression()
model2.fit(X_train2,y_train)
y_pred = model2.predict(X_val2)
print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades)


r^2: 0.019578236025706963 MAE: 0.06724077812682644 ROI lift: 1.0681719482028555


In [168]:
# trying z normaliztion 

X_val3 = X_val - X_val.mean(axis=0)/X_val.std(axis=0)
X_train3 = X_train - X_train.mean(axis=0)/X_train.std(axis=0)

model3 = LinearRegression()
model3.fit(X_train3,y_train)

y_pred = model3.predict(X_val3)

net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


C:\Users\GiladTK\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- realised_return

  warnings.warn(message, FutureWarning)


ValueError: X has 62 features, but LinearRegression is expecting 63 features as input.

In [94]:
# trying single feature selection 
r2_dict = {}
for feature in X_train.columns:
    X_train_feature = X_train[feature].values.reshape(-1, 1)  
    X_val_feature = X_val[feature].values.reshape(-1, 1)      

    model = LinearRegression()
    model.fit(X_train_feature, y_train)
    y_pred = model.predict(X_val_feature)
    r2_dict[feature] = r2_score(y_val, y_pred)
    


In [95]:
sorted_dict = sorted(r2_dict.items(), key=lambda x: x[1], reverse=True)
selected_keys = []
selected_keys2 = []

for key, value in r2_dict.items():
    if value > 0.0001:
        selected_keys.append(key)

for key, value in r2_dict.items():
    if value > 0.001:
        selected_keys2.append(key)

In [96]:
sorted_dict

[('int_rate', 0.01246158885772175),
 ('sub_grade', 0.01119979816875527),
 ('acc_open_past_24mths', 0.00621863684623325),
 ('inq_last_6mths', 0.004383352225970749),
 ('num_tl_op_past_12m', 0.0042929152257773495),
 ('open_il_24m', 0.003907008853120675),
 ('open_rv_24m', 0.003763225672135162),
 ('open_il_12m', 0.003736940060338201),
 ('open_acc_6m', 0.003372864504100792),
 ('inq_last_12m', 0.0032773201596226764),
 ('mths_since_recent_inq', 0.003089876459937746),
 ('inq_fi', 0.002563718687378702),
 ('RENT or ANY', 0.0024500533977794836),
 ('Verified', 0.002404559203037726),
 ('MORTGAGE', 0.0022681934596406883),
 ('fico_range_high', 0.002086970083480355),
 ('open_rv_12m', 0.00200814105961189),
 ('mort_acc', 0.0018452251648778395),
 ('avg_cur_bal', 0.001585492276680034),
 ('tot_hi_cred_lim', 0.0014835225611444791),
 ('mths_since_recent_bc', 0.0014760869357012263),
 ('dti', 0.0012712766797714492),
 ('initial_list_status', 0.0012546185979501745),
 ('mo_sin_rcnt_tl', 0.001170251661823829),
 ('N

In [97]:
X_train_small = X_train[selected_keys]
X_val_small = X_val[selected_keys]
model_s = LinearRegression()
model_s.fit(X_train_small,y_train)
y_pred = model_s.predict(X_val_small)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


r^2: 0.021949662382605983 MAE: 0.06776655997141422 ROI lift: 1.1418144893859499 net roi: 0.016043907452438742


In [98]:
X_train_small = X_train[selected_keys2]
X_val_small = X_val[selected_keys2]
model_s = LinearRegression()
model_s.fit(X_train_small,y_train)
y_pred = model_s.predict(X_val_small)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


r^2: 0.021293483333231555 MAE: 0.06776727226241591 ROI lift: 1.136826002022147 net roi: 0.015886093069002148


In [99]:
from sklearn.preprocessing import PolynomialFeatures

In [100]:
# lets try polynom 2 degree
poly = PolynomialFeatures(2)

X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.fit_transform(X_val)

model6 = LinearRegression()
model6.fit(X_train_poly,y_train)

y_pred = model6.predict(X_val_poly)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)

# weired as fuck !!!!!!!!!!!!! 

# ROI is bigger but r^2 is smaller 



r^2: 0.015042739686640494 MAE: 0.06807441970804161 ROI lift: 1.114745940533435 net roi: 0.015491142498403401


In [101]:
# lets try polynom 3 degree
# if its too big it wont even work...
X_train_small = X_train[selected_keys2]
X_val_small = X_val[selected_keys2]

poly = PolynomialFeatures(3)

X_train_poly = poly.fit_transform(X_train_small)
X_val_poly = poly.fit_transform(X_val_small)

model6 = LinearRegression()
model6.fit(X_train_poly,y_train)

y_pred = model6.predict(X_val_poly)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)

# weired as fuck !!!!!!!!!!!!! 

# ROI is bigger but r^2 is smaller 



r^2: 0.016638942346919205 MAE: 0.06782137822487698 ROI lift: 1.1221059855920497 net roi: 0.015935066285250123


In [102]:
from sklearn.preprocessing import RobustScaler

In [103]:
# this should limit outliers
robost = RobustScaler()

X_train_limited = robost.fit_transform(X_train)
X_val_limited = robost.fit_transform(X_val)

model7 = LinearRegression()
model7.fit(X_train_limited,y_train)

y_pred = model7.predict(X_val_limited)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


r^2: 0.021030584940133434 MAE: 0.06826619891079688 ROI lift: 1.1439812419963207 net roi: 0.01622631662583642


In [104]:
# lets go bask a little and try without grade 

# start of modeling process 

X_train9 = X_train.drop(columns="sub_grade")
X_val9 = X_val.drop(columns="sub_grade")

model9 = LinearRegression()
model9.fit(X_train9,y_train)

y_pred = model9.predict(X_val9)
net_ROI = y_test[y_pred >= 0.02].mean()

print("r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)

# no change - i recommend leave it out

r^2: 0.0210227009731081 MAE: 0.06794546178924477 ROI lift: 1.1281959428808008 net roi: 0.016260195150355438


# from now on im using k folds 
* using all data for train and test

In [105]:
from sklearn.model_selection import KFold

k = 5  
kf = KFold(n_splits=k, shuffle=True, random_state=99)

r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # later use (needs to add months)
    #amounts = payments[test_index]
    
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
    
    r2 += r2_score(y_test,y_pred)
    MAE += mean_absolute_error(y_test,y_pred)
    ROI_lift += y_test[y_pred >= 0.02].mean()/ROI_grades
    net_ROI += y_test[y_pred >= 0.02].mean()

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)


r^2: 0.02264188568852219 MAE: 0.06756144919081794 ROI lift: 1.132510946125894 net_roi: 0.02842377584212181


In [106]:
r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    

# trying min max normaliztion 
    X_test = (X_test-X_test.min(axis=0))/(X_test.max(axis=0)-X_test.min(axis=0))
    X_train = (X_train-X_train.min(axis=0))/(X_train.max(axis=0)-X_train.min(axis=0))

    model2 = LinearRegression()
    model2.fit(X_train,y_train)
    y_pred = model2.predict(X_test)
    
    r2 += r2_score(y_test,y_pred)
    MAE += mean_absolute_error(y_test,y_pred)
    ROI_lift += y_test[y_pred >= 0.02].mean()/ROI_grades
    net_ROI += y_test[y_pred >= 0.02].mean()

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)


r^2: 0.014812833761140709 MAE: 0.06750793743460301 ROI lift: 1.0857438703907305 net_roi: 0.027250015109799453


In [107]:
r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
# trying z normaliztion 

    X_test = X_test - X_test.mean(axis=0)/X_test.std(axis=0)
    X_train = X_train - X_train.mean(axis=0)/X_train.std(axis=0)

    model3 = LinearRegression()
    model3.fit(X_train,y_train)

    y_pred = model3.predict(X_test)

    r2 += r2_score(y_test,y_pred)
    MAE += mean_absolute_error(y_test,y_pred)
    ROI_lift += y_test[y_pred >= 0.02].mean()/ROI_grades
    net_ROI += y_test[y_pred >= 0.02].mean()

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)


r^2: 0.015202611082667716 MAE: 0.06816223947560586 ROI lift: 1.1356388609700359 net_roi: 0.028502280293392034


In [108]:
r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
# trying z normaliztion 

    X_test = X_test - X_test.mean(axis=0)/X_test.std(axis=0)
    X_train = X_train - X_train.mean(axis=0)/X_train.std(axis=0)

    model3 = LinearRegression()
    model3.fit(X_train,y_train)

    y_pred = model3.predict(X_test)

    r2 += r2_score(y_test,y_pred)
    MAE += mean_absolute_error(y_test,y_pred)
    ROI_lift += y_test[y_pred >= 0.02].mean()/ROI_grades
    net_ROI += y_test[y_pred >= 0.02].mean()

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)


r^2: 0.015202611082667716 MAE: 0.06816223947560586 ROI lift: 1.1356388609700359 net_roi: 0.028502280293392034


In [109]:
poly = PolynomialFeatures(2)

X_poly = poly.fit_transform(X)

In [110]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X_poly):
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Replace inf values with 0
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 += r2_score(y_test, y_pred)
    MAE += mean_absolute_error(y_test, y_pred)
    net_ROI += y_test[y_pred >= 0.02].mean()
    ROI_lift += y_test[y_pred >= 0.02].mean() / ROI_grades
    

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)
 

r^2: 0.020393656340805876 MAE: 0.0675932140488117 ROI lift: 1.1430388437335108 net_roi: 0.028688005166095553


In [111]:
# r2 = 0 
# MAE = 0 
# ROI_lift = 0
# net_ROI = 0

# # lets try some combinations 

# poly = PolynomialFeatures(2)

# X_poly = poly.fit_transform(X)
# for train_index, test_index in kf.split(X_poly):
    
#     X_train, X_test = X_poly[train_index], X_poly[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    
#     X_test = X_test - X_test.mean(axis=0)/X_test.std(axis=0)
#     X_train = X_train - X_train.mean(axis=0)/X_train.std(axis=0)

#     model = LinearRegression()
#     model.fit(X_train,y_train)

#     y_pred = model.predict(X_test)

#     r2 += r2_score(y_test,y_pred)
#     MAE += mean_absolute_error(y_test,y_pred)
#     ROI_lift += y_test[y_pred >= 0.02].mean()/ROI_grades

# print("r^2:",r2/5,"MAE:",MAE/5,"ROI lift:",ROI_lift/5)

In [112]:
def limit_outlier_iqr(X):
    Q1 = np.percentile(X, 25)
    Q3 = np.percentile(X, 75)
    IQR=Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    X_clipped = np.clip(X, lower_bound, upper_bound)

    return X_clipped

In [113]:
# 1.5 IQR method for limit outliaers 
X_limited = limit_outlier_iqr(X)

r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X_limited):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model3 = LinearRegression()
    model3.fit(X_train,y_train)

    y_pred = model3.predict(X_test)

    r2 += r2_score(y_test, y_pred)
    MAE += mean_absolute_error(y_test, y_pred)
    net_ROI += y_test[y_pred >= 0.02].mean()
    ROI_lift += y_test[y_pred >= 0.02].mean() / ROI_grades
    

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)
 

r^2: 0.02264188568852219 MAE: 0.06756144919081794 ROI lift: 1.132510946125894 net_roi: 0.02842377584212181


In [114]:
# 1.5 IQR method for limit outliaers 
r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X_limited):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # trying z normaliztion 

    X_test = X_test - X_test.mean(axis=0)/X_test.std(axis=0)
    X_train = X_train - X_train.mean(axis=0)/X_train.std(axis=0)

    model3 = LinearRegression()
    model3.fit(X_train,y_train)

    y_pred = model3.predict(X_test)

    r2 += r2_score(y_test, y_pred)
    MAE += mean_absolute_error(y_test, y_pred)
    net_ROI += y_test[y_pred >= 0.02].mean()
    ROI_lift += y_test[y_pred >= 0.02].mean() / ROI_grades
    

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)
 

r^2: 0.015202611082667716 MAE: 0.06816223947560586 ROI lift: 1.1356388609700359 net_roi: 0.028502280293392034


In [115]:
poly = PolynomialFeatures(2)

X_poly_limited_selected = poly.fit_transform(X_limited[selected_keys2])

In [116]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X_poly_limited_selected):
    X_train, X_test = X_poly_limited_selected[train_index], X_poly_limited_selected[test_index]
    y_train, y_test = y[train_index], y[test_index]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Replace inf values with 0
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 += r2_score(y_test, y_pred)
    MAE += mean_absolute_error(y_test, y_pred)
    net_ROI += y_test[y_pred >= 0.02].mean()
    ROI_lift += y_test[y_pred >= 0.02].mean() / ROI_grades
    

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)
 

r^2: 0.022502692632205834 MAE: 0.06727781374109303 ROI lift: 1.0962603759338123 net_roi: 0.027513958515575383


In [117]:
from sklearn.ensemble import AdaBoostRegressor

In [118]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [119]:
from sklearn.neighbors import KNeighborsRegressor

In [120]:
for n in range(3,50,5):

    knn_model_simple = KNeighborsRegressor(n_neighbors=n)
    knn_model_simple.fit(X_train,y_train)
    y_pred = knn_model_simple.predict(X_test)
    
    net_ROI = y_test[y_pred >= 0.02].mean()

    print("i","r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)

    

ValueError: Found input variables with inconsistent numbers of samples: [53244, 53243]

In [ ]:
knn_model_simple = KNeighborsRegressor(n_neighbors=5)
knn_model_simple.fit(X_train,y_train)
y_pred = knn_model_simple.predict(X_test)
        
net_ROI = y_test[y_pred >= 0.02].mean()

print("i","r^2:",r2_score(y_val,y_pred),"MAE:",mean_absolute_error(y_val,y_pred),"ROI lift:",y_val[y_pred >= 0.02].mean()/ROI_grades,"net roi:",net_ROI)


In [ ]:
from skl

In [166]:
from sklearn.model_selection import KFold

k = 5  
kf = KFold(n_splits=k, shuffle=True, random_state=99)
y100 = y*100
ROI_grades = ROI_grades*100
r2 = 0 
MAE = 0 
ROI_lift = 0
net_ROI = 0

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y100.iloc[train_index], y100.iloc[test_index]
    table = 
    # later use (needs to add months)
    #amounts = payments[test_index]
    
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
    
    r2 += r2_score(y_test,y_pred)
    MAE += mean_absolute_error(y_test,y_pred)
    
    ROI_lift += y_test[y_pred >= 2].mean()/ROI_grades
    net_ROI += y_test[y_pred >= 2].mean()

print("r^2:", r2/5, "MAE:", MAE/5, "ROI lift:", ROI_lift/5,"net_roi:",net_ROI/5)


r^2: 1.0 MAE: 3.1352172993102575e-12 ROI lift: 0.0226748526338039 net_roi: 5.69093774078844


In [ ]:
len(selected_keys)

In [ ]:
len(selected_keys2)